In [2]:
'''
version 1.0
In version 1.0, I only implemented the game of evading enemies. 
There was no Q-learning, and the game optimization was poor. 
It was extremely difficult to quit the game, and it often became unresponsive.
'''
import pygame
import random
import sys

pygame.init()

WIDTH = 800
HEIGHT = 600

RED = (255,0,0)
BLUE = (0,0,255)
YELLOW = (255,255,0)
BACKGROUND_COLOR = (255,255,255)

player_size = 50
player_pos = [WIDTH/2, HEIGHT-player_size]
playerstep = 50;

enemy_size = 50
enemy_pos = [random.randint(0,WIDTH-enemy_size), 0]
enemy_list = [enemy_pos]

SPEED = 30

screen = pygame.display.set_mode((WIDTH, HEIGHT))

game_over = False

score = 0

clock = pygame.time.Clock()

myFont = pygame.font.SysFont("monospace", 35)

enemy_image = pygame.image.load("boom.png").convert_alpha()
enemy_image = pygame.transform.scale(enemy_image, (enemy_size, enemy_size))
player_image = pygame.image.load("player.png").convert_alpha()
player_image = pygame.transform.scale(player_image, (player_size, player_size)) 

def set_level(score, SPEED):
	if score < 20:
		SPEED = 2
	elif score < 40:
		SPEED = 4
	elif score < 60:
		SPEED = 8
	else:
		SPEED = 12
	return SPEED

def drop_enemies(enemy_list):
	delay = random.random()
	if len(enemy_list) < 10 and delay < 0.1:
		x_pos = random.randint(0,WIDTH-enemy_size)
		y_pos = 0
		enemy_list.append([x_pos, y_pos])

def draw_enemies(enemy_list):
	for enemy_pos in enemy_list:
		screen.blit(enemy_image, (enemy_pos[0], enemy_pos[1]))
        
def update_enemy_positions(enemy_list, score):
	for idx, enemy_pos in enumerate(enemy_list):
		if enemy_pos[1] >= 0 and enemy_pos[1] < HEIGHT:
			enemy_pos[1] += SPEED
		else:
			enemy_list.pop(idx)
			score += 1
	return score

def collision_check(enemy_list, player_pos):
	for enemy_pos in enemy_list:
		if detect_collision(enemy_pos, player_pos):
			return True
	return False

def detect_collision(player_pos, enemy_pos):
	p_x = player_pos[0]
	p_y = player_pos[1]

	e_x = enemy_pos[0]
	e_y = enemy_pos[1]

	if (e_x >= p_x and e_x < (p_x + player_size)) or (p_x >= e_x and p_x < (e_x+enemy_size)):
		if (e_y >= p_y and e_y < (p_y + player_size)) or (p_y >= e_y and p_y < (e_y+enemy_size)):
			return True
	return False

while not game_over:

	for event in pygame.event.get():
		if event.type == pygame.QUIT:
			sys.exit()

		if event.type == pygame.KEYDOWN:

			x = player_pos[0]
			y = player_pos[1]

			if event.key == pygame.K_LEFT:
				x -= playerstep
			elif event.key == pygame.K_RIGHT:
				x += playerstep

			player_pos = [x,y]

	screen.fill(BACKGROUND_COLOR)

	drop_enemies(enemy_list)
	score = update_enemy_positions(enemy_list, score)

	text = "Score:" + str(score)
	label = myFont.render(text, 1, YELLOW)
	screen.blit(label, (WIDTH-200, HEIGHT-40))

	if collision_check(enemy_list, player_pos):
		game_over = True

	draw_enemies(enemy_list)
	screen.blit(player_image, (player_pos[0], player_pos[1]))

	clock.tick(30)

	pygame.display.update()


pygame 2.6.1 (SDL 2.28.4, Python 3.12.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


NameError: name 'SPEED' is not defined

In [2]:
'''
version 2.0
In version 2.0, I incorporated Q - learning. 
I optimized the level settings. The reason for setting up levels was to help the program with pre-training to achieve better results. (In the tests, through optimizing the level settings, the scores did increase. In previous tests, the score per attempt did not exceed 30, but afterwards it could basically exceed 30.) 
At the same time, I added game hints and optimized the quit system.
'''
import pygame
import random
import sys
import numpy as np
from pygame.locals import *
from initializers import *
from classes import *

class State:
    def __init__(self, player, enemy):
        self.player = player
        self.enemy = enemy

pygame.init()

WIDTH = 800
HEIGHT = 600
screen = pygame.display.set_mode((WIDTH, HEIGHT))

RED = (255, 0, 0)
YELLOW = (255, 255, 0)
BACKGROUND_COLOR = (255, 255, 255)
BLACK = (0,0,0)
player_size = 50
enemy_size = 50
SPEED = 10

myFont = pygame.font.SysFont("monospace", 35, bold=True)
current_stage = "First Stage"

# load image
enemy_image = pygame.image.load("boom.png").convert_alpha()
enemy_image = pygame.transform.scale(enemy_image, (enemy_size, enemy_size))
player_image = pygame.image.load("player.png").convert_alpha()
player_image = pygame.transform.scale(player_image, (player_size, player_size))

score = 0
deaths = 0
total_score = 0
death_limit = 3
level = 1
game_over = False

# Q-learning parameter
lr = 0.85  # learning rate
y = 0.99   # discount rate
epsilon = 1.0 
epsilon_decay = 0.995  
min_epsilon = 0.1  

# initial Q-table
states = (WIDTH // player_size) * (HEIGHT // enemy_size)  
actions = 3 
Q = np.zeros((states, actions))

clock = pygame.time.Clock()

def get_state(player, enemy):
    player_x = player.x // player_size
    enemy_y = enemy.top // enemy_size
    return player_x + enemy_y * (WIDTH // player_size)

def new_state_after_action(s, act):
    updatedPlayer = None

    if act == 2:  # right
        if s.player.right + s.player.width > WIDTH:
            updatedPlayer = s.player
        else:
            updatedPlayer = pygame.Rect(s.player.left + s.player.width, s.player.top, s.player.width,
                          s.player.height)
    elif act == 1:  # left
        if s.player.left - s.player.width < 0:
            updatedPlayer = s.player
        else:
            updatedPlayer = pygame.Rect(s.player.left - s.player.width, s.player.top, s.player.width,
                          s.player.height)
    else:  # stay here
        updatedPlayer = s.player

    s.enemy.top += SPEED
    updatedEnemy = s.enemy
    return State(updatedPlayer, updatedEnemy)

def calculate_reward(player, enemy):
    if(player.top <= enemy.bottom <= player.bottom):
        if not (player.left <= enemy.left <= player.right):
            reward = 1
        else:
            reward = -1
        enemy.left = random.randint(0, WIDTH - enemy_size)
        enemy.top = 0
    else:
        reward = 0
        enemy.top += SPEED
    return reward

def draw_player(player):
    screen.blit(player_image, (player.x, player.y))

def draw_enemies(enemy):
    screen.blit(enemy_image, (enemy.x, enemy.y))

def display_text(text, position, color):
    label = myFont.render(text, True, color)
    text_rect = label.get_rect(center=position)
    screen.blit(label, text_rect)

while not game_over:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()

    screen.fill(BACKGROUND_COLOR)

    state = get_state(player, enemy)

    # use ε-greedy to select best action
    if random.uniform(0, 1) < epsilon:
        act = random.randint(0, 2)  
    else:
        act = np.argmax(Q[state])  # select the highest Q value action

    s = State(player, enemy)
    r0 = calculate_reward(s.player, s.enemy)
    s1 = new_state_after_action(s, act)

    player = s1.player
    enemy = s1.enemy

    # renew score and death
    if r0 == 1:  
        score += r0
        total_score += r0
    elif r0 == -1:  
        deaths += 1

    if deaths > death_limit:
        game_over = True

    # renew Q-learning
    next_state = get_state(player, enemy)
    max_future_q = np.max(Q[next_state])  
    current_q = Q[state][act] 
    # most important part of Q-learning 
    Q[state][act] = current_q + lr * (r0 + y * max_future_q - current_q)

    # renew epsilon
    if epsilon > min_epsilon:
        epsilon *= epsilon_decay

    # renew level
    if level == 1 and deaths <= death_limit and score > 10:
        level += 1
        deaths = 0
        score = 0
        SPEED += 5
        current_stage = "Second Stage"
        print(current_stage)
    elif level == 2 and deaths <= death_limit and score > 30:
        level += 1
        deaths = 0
        score = 0
        SPEED += 5
        current_stage = "Final Stage"
        print(current_stage)
    elif level == 3 and deaths <= death_limit and score > 100:
        print("Congratulations")
        game_over = True

    display_text(current_stage, (300, 50), BLACK)

    # display
    text = "Score: " + str(score)
    display_text(text, (WIDTH - 200, HEIGHT - 40), YELLOW)

    text = "Deaths: " + str(deaths)
    display_text(text, (WIDTH - 200, HEIGHT - 80), RED)

    draw_enemies(enemy)
    draw_player(player)

    clock.tick(30)
    pygame.display.update()

# hint game over
screen.fill(BACKGROUND_COLOR)
game_over_text = "Game Over! Final Score:"
display_text(game_over_text, (WIDTH / 2, HEIGHT / 2 - 20), RED)

total_score_text = "Total Score: " + str(total_score)
display_text(total_score_text, (WIDTH / 2, HEIGHT / 2 + 20), RED)

pygame.display.update()

#quit system
while True:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()

pygame 2.6.1 (SDL 2.28.4, Python 3.12.4)
Hello from the pygame community. https://www.pygame.org/contribute.html
Second Stage


SystemExit: 

E:\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [7]:
'''
version 3.0
In version 3.0, I added the "supply" class to enhance the gameplay. 
Otherwise, sometimes the player would be seen stuck against the wall, motionless beside it.
'''
import pygame
import random
import sys
import numpy as np
from pygame.locals import *

pygame.init()

WIDTH = 800
HEIGHT = 600
screen = pygame.display.set_mode((WIDTH, HEIGHT))

RED = (255, 0, 0)
YELLOW = (255, 255, 0)
BACKGROUND_COLOR = (255, 255, 255)
BLACK = (0, 0, 0)
player_size = 50
enemy_size = 50
supply_size = 50
SPEED = 10

myFont = pygame.font.SysFont("monospace", 35, bold=True)
current_stage = "First Stage"

# load image
enemy_image = pygame.image.load("boom.png").convert_alpha()
enemy_image = pygame.transform.scale(enemy_image, (enemy_size, enemy_size))
player_image = pygame.image.load("player.png").convert_alpha()
player_image = pygame.transform.scale(player_image, (player_size, player_size))
supply_image = pygame.image.load("supply.jpg").convert_alpha()
supply_image = pygame.transform.scale(supply_image, (supply_size, supply_size))

score = 0
deaths = 0
total_score = 0
death_limit = 3
level = 1
game_over = False

# Q-learning parameter
lr = 0.85  # learning rate
y = 0.99  # discount rate
epsilon = 1.0  
epsilon_decay = 0.995 
min_epsilon = 0.1  

# initial Q-table
states = (WIDTH // player_size) * (HEIGHT // enemy_size)  
actions = 3  
Q = np.zeros((states, actions))

clock = pygame.time.Clock()

# class State 
class State:
    def __init__(self, player, enemy):
        self.player = player
        self.enemy = enemy

# class Supply
class Supply:
    def __init__(self, x, y, size):
        self.rect = pygame.Rect(x, y, size, size)
        self.image = supply_image

supply_list = []

def get_state(player, enemy):
    player_x = player.x // player_size
    enemy_y = enemy.top // enemy_size
    return player_x + enemy_y * (WIDTH // player_size)

def new_state_after_action(s, act):
    updatedPlayer = None
    if act == 2:  # right
        if s.player.right + s.player.width > WIDTH:
            updatedPlayer = s.player
        else:
            updatedPlayer = pygame.Rect(s.player.left + s.player.width, s.player.top, s.player.width, s.player.height)
    elif act == 1:  # left
        if s.player.left - s.player.width < 0:
            updatedPlayer = s.player
        else:
            updatedPlayer = pygame.Rect(s.player.left - s.player.width, s.player.top, s.player.width, s.player.height)
    else:  # stay here
        updatedPlayer = s.player

    s.enemy.top += SPEED
    updatedEnemy = s.enemy
    return State(updatedPlayer, updatedEnemy)

def calculate_reward(player, enemy):
    reward = 0
    if player.top <= enemy.bottom <= player.bottom:
        if not (player.left <= enemy.left <= player.right):
            reward = 1
        else:
            reward = -1
        enemy.left = random.randint(0, WIDTH - enemy_size)
        enemy.top = 0
    else:
        reward = 0
    enemy.top += SPEED

    # check collisions with supply
    for supply in supply_list[:]:
        if player.colliderect(supply.rect):
            reward += 5
            supply_list.remove(supply)

    return reward

def draw_player(player):
    screen.blit(player_image, (player.x, player.y))

def draw_enemies(enemy):
    screen.blit(enemy_image, (enemy.x, enemy.y))

def draw_supplies():
    for supply in supply_list:
        screen.blit(supply.image, supply.rect.topleft)

def display_text(text, position, color):
    label = myFont.render(text, True, color)
    text_rect = label.get_rect(center=position)
    screen.blit(label, text_rect)

# randomly generate supply
def generate_supply():
    if random.uniform(0, 1) < 0.01:  # 1% generate supply
        x = random.randint(0, WIDTH - supply_size)
        y = 0
        supply_list.append(Supply(x, y, supply_size))

# renew supply location
def update_supplies():
    for supply in supply_list[:]:
        supply.rect.top += SPEED  
        if supply.rect.top > HEIGHT: 
            supply_list.remove(supply)

def check_supply_collision(player):
    global score, total_score
    for supply in supply_list[:]:
        if player.colliderect(supply.rect):
            score += 5
            total_score += 5
            supply_list.remove(supply)

player = pygame.Rect(WIDTH // 2, HEIGHT - player_size, player_size, player_size)
enemy = pygame.Rect(random.randint(0, WIDTH - enemy_size), 0, enemy_size, enemy_size)

while not game_over:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()

    screen.fill(BACKGROUND_COLOR)

    generate_supply()

    update_supplies()

    state = get_state(player, enemy)

    # use ε-greedy to select best action
    if random.uniform(0, 1) < epsilon:
        act = random.randint(0, 2)  
    else:
        act = np.argmax(Q[state])

    s = State(player, enemy)
    r0 = calculate_reward(s.player, s.enemy)
    s1 = new_state_after_action(s, act)

    player = s1.player
    enemy = s1.enemy

    check_supply_collision(player)

    if r0 == 1:  
        score += r0
        total_score += r0
    elif r0 == -1:  
        deaths += 1

    if deaths > death_limit:
        game_over = True

    # renew Q-learning 
    next_state = get_state(player, enemy)
    max_future_q = np.max(Q[next_state])  
    current_q = Q[state][act]  
    # the most important part of Q-learning
    Q[state][act] = current_q + lr * (r0 + y * max_future_q - current_q)

    # renew epsilon
    if epsilon > min_epsilon:
        epsilon *= epsilon_decay

    # renew level
    if level == 1 and deaths <= death_limit and score > 30:
        level += 1
        deaths = 0
        score = 0
        SPEED += 5
        current_stage = "Second Stage"
        print(current_stage)
    elif level == 2 and deaths <= death_limit and score > 70:
        level += 1
        deaths = 0
        score = 0
        SPEED += 5
        current_stage = "Final Stage"
        print(current_stage)
    elif level == 3 and deaths <= death_limit and score > 100:
        print("Congratulations")
        game_over = True

    display_text(current_stage, (300, 50), BLACK)

    text = "Score: " + str(score)
    display_text(text, (WIDTH - 200, HEIGHT - 40), YELLOW)

    text = "Deaths: " + str(deaths)
    display_text(text, (WIDTH - 200, HEIGHT - 80), RED)

    draw_enemies(enemy)
    draw_player(player)
    draw_supplies()

    clock.tick(30)
    pygame.display.update()

# hint gameover
screen.fill(BACKGROUND_COLOR)
game_over_text = "Game Over! Final Score:"
display_text(game_over_text, (WIDTH / 2, HEIGHT / 2 - 20), RED)

total_score_text = "Total Score: " + str(total_score)
display_text(total_score_text, (WIDTH / 2, HEIGHT / 2 + 20), RED)

pygame.display.update()

# quit system
while True:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()

Second Stage


SystemExit: 